In [204]:
from bs4 import BeautifulSoup
import requests
import time

In [40]:
headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
html_page = requests.get('https://www.trustpilot.com/review/hellofresh.com', headers = headers)
soup = BeautifulSoup(html_page.content, 'html.parser')

In [288]:
s = soup.prettify

In [76]:
content = soup.find('div', class_ = "styles_reviewContent__0Q2Tg")

In [77]:
content.find('p').text

"Horrible company to deal, impossible to cancel an order that hasn't even happened yet. They charge your card even if you do manage to cancel your order. No reminders at all about upcoming meals in case you wanted to cancel or suspend deliveries. Customer service on the phone is horrendous all they say is I can't do that, they'll never grab a manager to help fix the problem. Horrible Horrible company"

In [289]:
new = soup.findAll('div', class_ = "styles_reviewCardInner__EwDq2")

In [290]:
text = soup.findAll('p', class_ = 'typography_body-l__KUYFJ')

In [236]:
def scrape_review_pages(page_start, page_end):
    headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    reviews = []
    for i in range(page_start, page_end):
        url = 'https://www.trustpilot.com/review/hellofresh.com?page=' + str(i)
        try:
            html_page = requests.get(url, headers = headers)
            #print(url)
            html_page.raise_for_status()
            soup = BeautifulSoup(html_page.content, 'html.parser')
            review_elements = soup.find_all("article", class_="styles_reviewCard__hcAvl")
            for review_element in review_elements:
                review_text = review_element.find("p", class_="typography_body-l__KUYFJ")
                #print(review_text)
                if review_text is not None:
                    review_text = review_text.text.strip()
                else:
                    review_text = "No review text available."
                review_score_tag = review_element.find("div", class_="star-rating_starRating__4rrcf")
                if review_score_tag is not None:
                    review_score = int(review_score_tag.img['alt'].split()[1])
                else:
                    review_score = 0
                #print(review_score)
                review_object = {"text": review_text, "score": review_score}
                reviews.append(review_object)
            time.sleep(3)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {i}: {e}")
            continue
    return reviews

In [222]:
Hello_fresh_reviews = scrape_review_pages(2,275)
hf2 = scrape_review_pages(275,447)
hf3 = scrape_review_pages(447, 597)
hf4 = scrape_review_pages(597, 747)
hf5 = scrape_review_pages(747,900)
hf6 = scrape_review_pages(900,1100)
hf7 = scrape_review_pages(1100,1275)
hf8 = scrape_review_pages(1275,1475)
hf9 = scrape_review_pages(1475,1675)
hf10 = scrape_review_pages(1675,1875)
hf11 = scrape_review_pages(1875,2075)
hf12 = scrape_review_pages(2075,2200)
hf13 = scrape_review_pages(2200,2346)

In [271]:
import pandas as pd
df = pd.DataFrame(Hello_fresh_reviews)
df2 = pd.DataFrame(hf2)
df3 = pd.DataFrame(hf3)
df4 = pd.DataFrame(hf4)
df5 = pd.DataFrame(hf5)
df6 = pd.DataFrame(hf6)
df7 = pd.DataFrame(hf7)
df8 = pd.DataFrame(hf8)
df9 = pd.DataFrame(hf9)
df10 = pd.DataFrame(hf10)
df11 = pd.DataFrame(hf11)
df12 = pd.DataFrame(hf12)
df13 = pd.DataFrame(hf13)

In [313]:
fresh_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13], 0)

In [314]:
hello_fresh_df = fresh_df.reset_index(drop = True)

In [315]:
hello_fresh_df

,text,score
0,Almost everything was included to make meals a...,4
1,The recipes taste good and the food is fresh. ...,4
2,I've tried so many of the meal kit companies. ...,5
3,Delivery was missed altogether. Out of my 31 m...,1
4,"The food is fresh, portioned nicely and there ...",4
...,...,...
46875,HelloFresh treated me horribly. I signed up f...,1
46876,I wasn't sure about trying this but thought si...,5
46877,"I won't bore you with all the details, but thi...",1
46878,I tried hello fresh for about one month. Two ...,2


In [316]:
hello_fresh_df[hello_fresh_df['text'] == "No review text available."].index

Int64Index([  121,   131,   135,   141,   149,   151,   154,   158,   163,
              165,
            ...
            42940, 42954, 42997, 43191, 43348, 44226, 45000, 45255, 45315,
            45731],
           dtype='int64', length=2352)

In [317]:
hello_fresh_df = hello_fresh_df.drop(hello_fresh_df[hello_fresh_df['text'] == "No review text available."].index)

In [318]:
hello_fresh_df = hello_fresh_df.reset_index(drop=True)

In [320]:
changing_numeric_tosentiment = {1 : "negative", 
                              2:"negative", 
                              3: "neutral", 
                              4: "positive", 
                              5: "positive" }
hello_fresh_df['score'] = hello_fresh_df['score'].replace(changing_numeric_tosentiment)

In [321]:
hello_fresh_df

,text,score
0,Almost everything was included to make meals a...,positive
1,The recipes taste good and the food is fresh. ...,positive
2,I've tried so many of the meal kit companies. ...,positive
3,Delivery was missed altogether. Out of my 31 m...,negative
4,"The food is fresh, portioned nicely and there ...",positive
...,...,...
44523,HelloFresh treated me horribly. I signed up f...,negative
44524,I wasn't sure about trying this but thought si...,positive
44525,"I won't bore you with all the details, but thi...",negative
44526,I tried hello fresh for about one month. Two ...,negative
